Imports necessary libraries: ray, torch, torch.nn, torchvision, torch.utils.data, and Ray Train-specific modules.

In [8]:
!pip install -q ray[default] torch torchvision tqdm ray[train]

In [4]:
from cmlextensions.ray_cluster import RayCluster

# Set up the Ray Cluster using CMLExtensions
cluster = RayCluster(num_workers=3, worker_cpu=4, worker_memory=8, worker_nvidia_gpu=1, head_cpu=4, head_memory=8, head_nvidia_gpu=1)
cluster.init()

Starting ray head...
Starting 3 ray workers...

--------------------
Ray cluster started
--------------------

The Ray dashboard is running at
https://nke47n3zmba7mpvh.goes-ocp-cml.apps.field-team-ocp-01.kcloud.cloudera.com/

To connect to this Ray cluster from this CML Session,
use the following Python code:
  import ray
  ray.init(address='ray://10.254.12.125:10001')



In [6]:
  import ray
  ray.init(address='ray://10.254.12.125:10001')

2025-04-04 08:00:12,943	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver
SIGTERM handler is not set because current thread is not the main thread.


Python version:,3.10.13
Ray version:,2.44.1
Dashboard:,http://127.0.0.1:8090


In [5]:
import os
from typing import Dict

import torch
from filelock import FileLock
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Normalize, ToTensor
from tqdm import tqdm

import ray.train
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer

2025-04-04 07:59:58,173	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-04 07:59:59,095	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [7]:
def get_dataloaders(batch_size):
    # Transform to normalize the input images
    transform = transforms.Compose([ToTensor(), Normalize((0.28604,), (0.32025,))])

    with FileLock(os.path.expanduser("~/data.lock")):
        # Download training data from open datasets
        training_data = datasets.FashionMNIST(
            root="~/data",
            train=True,
            download=True,
            transform=transform,
        )

        # Download test data from open datasets
        test_data = datasets.FashionMNIST(
            root="~/data",
            train=False,
            download=True,
            transform=transform,
        )

    # Create data loaders
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader


# Model Definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_func_per_worker(config: Dict):
    lr = config["lr"]
    epochs = config["epochs"]
    batch_size = config["batch_size_per_worker"]

    # Get dataloaders inside the worker training function
    train_dataloader, test_dataloader = get_dataloaders(batch_size=batch_size)

    # [1] Prepare Dataloader for distributed training
    # Shard the datasets among workers and move batches to the correct device
    # =======================================================================
    train_dataloader = ray.train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = ray.train.torch.prepare_data_loader(test_dataloader)

    model = NeuralNetwork()

    # [2] Prepare and wrap your model with DistributedDataParallel
    # Move the model to the correct GPU/CPU device
    # ============================================================
    model = ray.train.torch.prepare_model(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Model training loop
    for epoch in range(epochs):
        if ray.train.get_context().get_world_size() > 1:
            # Required for the distributed sampler to shuffle properly across epochs.
            train_dataloader.sampler.set_epoch(epoch)

        model.train()
        for X, y in tqdm(train_dataloader, desc=f"Train Epoch {epoch}"):
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        test_loss, num_correct, num_total = 0, 0, 0
        with torch.no_grad():
            for X, y in tqdm(test_dataloader, desc=f"Test Epoch {epoch}"):
                pred = model(X)
                loss = loss_fn(pred, y)

                test_loss += loss.item()
                num_total += y.shape[0]
                num_correct += (pred.argmax(1) == y).sum().item()

        test_loss /= len(test_dataloader)
        accuracy = num_correct / num_total

        # [3] Report metrics to Ray Train
        # ===============================
        ray.train.report(metrics={"loss": test_loss, "accuracy": accuracy})


def train_fashion_mnist(num_workers=2, use_gpu=False):
    global_batch_size = 32

    train_config = {
        "lr": 1e-3,
        "epochs": 10,
        "batch_size_per_worker": global_batch_size // num_workers,
    }

    # Configure computation resources
    scaling_config = ScalingConfig(num_workers=num_workers, use_gpu=use_gpu)

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=train_func_per_worker,
        train_loop_config=train_config,
        scaling_config=scaling_config,
    )

    # [4] Start distributed training
    # Run `train_func_per_worker` on all workers
    # =============================================
    result = trainer.fit()
    print(f"Training result: {result}")


In [9]:
# Connect to the existing Ray cluster
train_fashion_mnist(num_workers=3, use_gpu=True)

(TunerInternal pid=5392) AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


(TunerInternal pid=5392) 
(TunerInternal pid=5392) View detailed results here: /home/cdsw/ray_results/TorchTrainer_2025-04-04_09-08-58
(TunerInternal pid=5392) To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-04-04_07-58-21_269898_174/artifacts/2025-04-04_09-10-40/TorchTrainer_2025-04-04_09-08-58/driver_artifacts`


(TrainTrainable pid=524, ip=10.254.5.92) Trainable.setup took 10.083 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


(TunerInternal pid=5392) 
(TunerInternal pid=5392) Training started with configuration:
(TunerInternal pid=5392) ╭─────────────────────────────────────────────────╮
(TunerInternal pid=5392) │ Training config                                 │
(TunerInternal pid=5392) ├─────────────────────────────────────────────────┤
(TunerInternal pid=5392) │ train_loop_config/batch_size_per_worker      10 │
(TunerInternal pid=5392) │ train_loop_config/epochs                     10 │
(TunerInternal pid=5392) │ train_loop_config/lr                      0.001 │
(TunerInternal pid=5392) ╰─────────────────────────────────────────────────╯


(RayTrainWorker pid=609, ip=10.254.5.92) Setting up process group for: env:// [rank=0, world_size=3]
(TorchTrainer pid=524, ip=10.254.5.92) Started distributed worker processes: 
(TorchTrainer pid=524, ip=10.254.5.92) - (node_id=2016aa8d81bf705b555d3e38fcd3418265fc395aa3cac9605675967c, ip=10.254.5.92, pid=609) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=524, ip=10.254.5.92) - (node_id=8792d6a98db40c93ce9d013e5ff47bf7f8b36560afae053a2297343b, ip=10.254.7.86, pid=484) world_rank=1, local_rank=0, node_rank=1
(TorchTrainer pid=524, ip=10.254.5.92) - (node_id=517abdb83fdf3f9c595e10c94caef6af39bb85d020eac5ef97fcf5b4, ip=10.254.6.97, pid=559) world_rank=2, local_rank=0, node_rank=2
(RayTrainWorker pid=609, ip=10.254.5.92) Moving model to device: cuda:0
(RayTrainWorker pid=609, ip=10.254.5.92) Wrapping provided model in DistributedDataParallel.
(RayTrainWorker pid=559, ip=10.254.6.97) Moving model to device: cuda:0
(RayTrainWorker pid=559, ip=10.254.6.97) Wrapping provided model 

...

(TunerInternal pid=5392) 
(TunerInternal pid=5392) Training finished iteration 9 at 2025-04-04 09:16:48. Total running time: 6min 6s
(TunerInternal pid=5392) ╭───────────────────────────────╮
(TunerInternal pid=5392) │ Training result               │
(TunerInternal pid=5392) ├───────────────────────────────┤
(TunerInternal pid=5392) │ checkpoint_dir_name           │
(TunerInternal pid=5392) │ time_this_iter_s      13.2409 │
(TunerInternal pid=5392) │ time_total_s          245.037 │
(TunerInternal pid=5392) │ training_iteration          9 │
(TunerInternal pid=5392) │ accuracy              0.87343 │
(TunerInternal pid=5392) │ loss                  0.35364 │
(TunerInternal pid=5392) ╰───────────────────────────────╯


Test Epoch 9: 100%|██████████| 334/334 [00:00<00:00, 346.35it/s]


(TunerInternal pid=5392) 
(TunerInternal pid=5392) Training finished iteration 10 at 2025-04-04 09:17:01. Total running time: 6min 19s
(TunerInternal pid=5392) ╭───────────────────────────────╮
(TunerInternal pid=5392) │ Training result               │
(TunerInternal pid=5392) ├───────────────────────────────┤
(TunerInternal pid=5392) │ checkpoint_dir_name           │
(TunerInternal pid=5392) │ time_this_iter_s      13.0292 │
(TunerInternal pid=5392) │ time_total_s          258.067 │
(TunerInternal pid=5392) │ training_iteration         10 │
(TunerInternal pid=5392) │ accuracy              0.87283 │
(TunerInternal pid=5392) │ loss                  0.34522 │
(TunerInternal pid=5392) ╰───────────────────────────────╯
(TunerInternal pid=5392) 
(TunerInternal pid=5392) Training completed after 10 iterations at 2025-04-04 09:17:03. Total running time: 6min 20s


(TunerInternal pid=5392) Wrote the latest version of all result files and experiment state to '/home/cdsw/ray_results/TorchTrainer_2025-04-04_09-08-58' in 1.0291s.


(TunerInternal pid=5392) 
Training result: Result(
  metrics={'loss': 0.3452150629258932, 'accuracy': 0.8728254349130175},
  path='/home/cdsw/ray_results/TorchTrainer_2025-04-04_09-08-58/TorchTrainer_ae4a4_00000_0_2025-04-04_09-10-42',
  filesystem='local',
  checkpoint=None
)


In [3]:
cluster.terminate()